### Do the imports
- If I am doing any processing of files in a folder I like to import os and glob
- import cellpose models for training of inferance
- import something to read your images 
- import libraries to save your results


In [1]:
import os
import glob
import sys

import tifffile # read an write tiff files
from cellpose import models
import napari
import torch
import nd2
import numpy as np

from skimage.transform import rescale

import utils

Create the napari viewer to visualize images and mask

In [2]:
viewer = napari.Viewer()

Find the image files to work with. Different path for Window, Mac, and Linux

In [3]:
if sys.platform == 'darwin': 
    imagedir = "/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data"
elif sys.plaform == 'win32':
    imagedir = os.path.join(['S:', 'micro',
                             'internal', 'cjw',
                             'PythonImageProcessing',
                             'cellpose',
                             'Data'])
else:
   imagedir = "/n/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data"
   
imagefiles = sorted(glob.glob(f"{imagedir}/*.tif"))

for i, f in enumerate(imagefiles):
    print(i, os.path.basename(f))

0 01-yeast_many.tif
1 02-a01_42.tif
2 03-a01_43.tif
3 04-nuclei_fish.tif
4 05-yeast_01.tif
5 06-yeast_02.tif
6 C-hela-cells.tif
7 fluo_00.tif


In [8]:
file = 
xraw = tifffile.imread(imagefiles[1])
xraw.shape

(4, 2160, 2160)

Add the image to the napari viewer

In [7]:
viewer.layers.clear()
viewer.add_image(x, channel_axis=0)

[<Image layer 'Image' at 0x2dbf76590>,
 <Image layer 'Image [1]' at 0x2dbfea860>,
 <Image layer 'Image [2]' at 0x2fc06ccd0>,
 <Image layer 'Image [3]' at 0x2fc2b3460>]

In [36]:
x = xraw[[0, 3], 200:712, 200:712]
x.shape

(2, 512, 512)

### Setup the cellpose model

#### GPU setting
- on windows or linux with an NVIDIA GPU, set `gpu=True``
- on M1 or M2 mac, set `device=torch.device('mps')`
- on old Mac or Windows without GPU, set `gpu=False` -- this will be slower

#### Model
- cellpose has a number of pretrained models to use. Start with `cyto` or `cyto2` (even it is nuclei)
- info about pretrained cellpose [models](https://cellpose.readthedocs.io/en/latest/models.html)

In [41]:
d = torch.device('mps')

# change gpu=True if on windows, and get rid of device
# model = models.Cellpose(gpu=True, model_type='cyto2')
model = models.Cellpose(gpu=False, device=d, model_type='nuclei', net_avg=True)

### `model.eval`
Parameters
----------

The basic options:
- x : the image, can be a 2d numpy array, a list of numpy arrays,
or a 3d numpy array
- diameter : The approximate size of the object you are trying to segment
- channels :
    - [0, 0] for a grayscale image
    - [] 

Returns
-------
- masks : An array or list of arrays with segmenation labels/masks
- flows : A list of numpy arrays with fow

#### Try cellpose with no paramter changes

In [42]:

masks, flows, styles, diams = model.eval(x[1], channels=[0, 0],
                                         diameter=17)

Show the results from cellpose a layers in napari

In [43]:
viewer.layers.clear()
viewer.add_image(x, channel_axis=0)
viewer.add_labels(masks)


<Labels layer 'masks' at 0x3e3088070>

See what happens when the diameter is changed. What diameter is the best?

In [45]:
viewer.add_image(flows[0], visible=False, name="xyflow")
viewer.add_image(flows[2], visible=False, name="probability")

<Image layer 'probability [1]' at 0x38a8fed40>

In [56]:
masks2, flows2, styles2, diams2 = model.eval(x[1], channels=[0, 0],
                                         diameter=17,
                                         cellprob_threshold=-3,
                                         flow_threshold=.8)

In [58]:
#viewer.layers.clear()
#viewer.add_image(x, channel_axis=0)
if "masks2" in viewer.layers:
    viewer.layers.remove("mask2")
viewer.add_labels(masks2)

<Labels layer 'masks2' at 0x2e20eb610>

In [59]:
a = masks > 0
viewer.add_image(a, blending='additive')

<Image layer 'a' at 0x2e25f3f10>